<a href="https://colab.research.google.com/github/hariskusayum11/antKohYo/blob/main/AntTestRun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ant1 ระบบค้นหาเส้นทางเกาะยอ

In [ ]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
            self.pheromone = self.pheromone * self.decay
        return all_time_shortest_path

    def spread_pheromone(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))  # Going back to where we started
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)
        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move


def get_distance_matrix(selected_places, places, distances):
    indices = [places.index(place) for place in selected_places]
    selected_distances = distances[np.ix_(indices, indices)]
    return selected_distances, indices


def print_path_details(path, places, distances):
    print("\nเส้นทางที่ดีที่สุด:")
    total_distance = 0
    for i, (from_idx, to_idx) in enumerate(path):
        from_place = places[from_idx]
        to_place = places[to_idx]
        distance = distances[from_idx, to_idx]
        total_distance += distance
        print(f"เส้นทางที่ {i+1}: {from_place} ไป {to_place} ระยะทาง {distance:.2f} km")
    print(f"\nระยะทางรวม: {total_distance:.2f} km")


# ข้อมูลสถานที่และระยะทาง
places = [
    "สถาบันทักษิณคดีศึกษา", "สวนสวา", "วัดแหลมพ้อ", "วัดโคกเปี้ยว", "ศาลเจ้าไท้ก๋ง",
    "วัดท้ายยอ", "สำนักสงฆ์เขากุฎิ", "วัดเขาบ่อ", "ร้านค้า OTOP"
]

distances = np.array([
    [np.inf, 30.4, 4.0, 5.1, 5.5, 4.8, 3.0, 0.9, 3.3],
    [30.4, np.inf, 24.2, 26.5, 28.1, 27.4, 26.5, 27.2, 19.4],
    [4.0, 24.2, np.inf, 2.5, 4.1, 3.4, 2.9, 3.7, 1.5],
    [5.1, 26.5, 2.5, np.inf, 4.1, 3.4, 3.1, 3.7, 1.5],
    [5.5, 28.1, 4.1, 4.1, np.inf, 0.65, 4.4, 4.6, 3.1],
    [4.8, 27.4, 3.4, 3.4, 0.65, np.inf, 3.7, 3.9, 2.4],
    [3.0, 26.5, 2.9, 3.1, 4.4, 3.7, np.inf, 3.9, 2.4],
    [0.9, 27.2, 3.7, 3.7, 4.6, 3.9, 3.9, np.inf, 3.0],
    [3.3, 19.4, 1.5, 1.5, 3.1, 2.4, 2.4, 3.0, np.inf]
])


# ฟังก์ชันให้ผู้ใช้เลือกสถานที่
def user_select_places(places):
    selected_places = []
    print("กรุณาเลือกสถานที่จากรายการด้านล่าง (ป้อนหมายเลขที่ต้องการเลือก):")
    for i, place in enumerate(places):
        print(f"{i + 1}. {place}")

    print("ป้อนหมายเลขของสถานที่ที่ต้องการเลือก (คั่นด้วย comma ',' เช่น 1,2,3) หรือ 'q' เพื่อหยุด:")
    user_input = input("> ")

    if user_input.lower() == 'q':
        return selected_places

    indices = user_input.split(',')
    for index in indices:
        try:
            place_index = int(index.strip()) - 1
            if 0 <= place_index < len(places):
                selected_places.append(places[place_index])
        except ValueError:
            continue

    return selected_places


def main():
    while True:
        selected_places = user_select_places(places)

        if len(selected_places) < 2:
            print("กรุณาเลือกสถานที่อย่างน้อย 2 แห่งเพื่อคำนวณเส้นทาง")
            continue

        selected_distances, indices = get_distance_matrix(selected_places, places, distances)

        # เรียกใช้อัลกอริทึม Ant Colony Optimization
        ant_colony = AntColony(selected_distances, n_ants=10, n_best=5, n_iterations=50, decay=0.95, alpha=1, beta=1)
        shortest_path = ant_colony.run()

        # แสดงผลลัพธ์
        shortest_path_indices = shortest_path[0]
        print_path_details(shortest_path_indices, selected_places, selected_distances)

        # ถามผู้ใช้ว่าต้องการคำนวณอีกครั้งหรือไม่
        user_input = input("\nคุณต้องการเลือกสถานที่ใหม่หรือไม่? (y/n): ").strip().lower()
        if user_input != 'y':
            print("ขอบคุณที่ใช้โปรแกรม!")
            break


if __name__ == "__main__":
    main()


กรุณาเลือกสถานที่จากรายการด้านล่าง (ป้อนหมายเลขที่ต้องการเลือก):
1. สถาบันทักษิณคดีศึกษา
2. สวนสวา
3. วัดแหลมพ้อ
4. วัดโคกเปี้ยว
5. ศาลเจ้าไท้ก๋ง
6. วัดท้ายยอ
7. สำนักสงฆ์เขากุฎิ
8. วัดเขาบ่อ
9. ร้านค้า OTOP
ป้อนหมายเลขของสถานที่ที่ต้องการเลือก (คั่นด้วย comma ',' เช่น 1,2,3) หรือ 'q' เพื่อหยุด:


KeyboardInterrupt: Interrupted by user

ant2 โชว์เส้นทางที่มดเดินทั้งหมด50รอบ

In [ ]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])

            # แสดงการเดินของมดในแต่ละรอบ
            print(f"\nรอบที่ {i+1}:")
            for j, (path, dist) in enumerate(all_paths):
                print(f"มดตัวที่ {j+1}: เส้นทาง {path} ระยะทาง {dist:.2f} km")

            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path

            self.pheromone = self.pheromone * self.decay

        return all_time_shortest_path

    def spread_pheromone(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))  # Going back to where we started
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)
        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move


def get_distance_matrix(selected_places, places, distances):
    indices = [places.index(place) for place in selected_places]
    selected_distances = distances[np.ix_(indices, indices)]
    return selected_distances, indices


def print_path_details(path, places, distances):
    print("\nเส้นทางที่ดีที่สุด:")
    total_distance = 0
    for i, (from_idx, to_idx) in enumerate(path):
        from_place = places[from_idx]
        to_place = places[to_idx]
        distance = distances[from_idx, to_idx]
        total_distance += distance
        print(f"เส้นทางที่ {i+1}: {from_place} ไป {to_place} ระยะทาง {distance:.2f} km")
    print(f"\nระยะทางรวม: {total_distance:.2f} km")


# ข้อมูลสถานที่และระยะทาง
places = [
    "สถาบันทักษิณคดีศึกษา", "สวนสวา", "วัดแหลมพ้อ", "วัดโคกเปี้ยว", "ศาลเจ้าไท้ก๋ง",
    "วัดท้ายยอ", "สำนักสงฆ์เขากุฎิ", "วัดเขาบ่อ", "ร้านค้า OTOP"
]

distances = np.array([
    [np.inf, 30.4, 4.0, 5.1, 5.5, 4.8, 3.0, 0.9, 3.3],
    [30.4, np.inf, 24.2, 26.5, 28.1, 27.4, 26.5, 27.2, 19.4],
    [4.0, 24.2, np.inf, 2.5, 4.1, 3.4, 2.9, 3.7, 1.5],
    [5.1, 26.5, 2.5, np.inf, 4.1, 3.4, 3.1, 3.7, 1.5],
    [5.5, 28.1, 4.1, 4.1, np.inf, 0.65, 4.4, 4.6, 3.1],
    [4.8, 27.4, 3.4, 3.4, 0.65, np.inf, 3.7, 3.9, 2.4],
    [3.0, 26.5, 2.9, 3.1, 4.4, 3.7, np.inf, 3.9, 2.4],
    [0.9, 27.2, 3.7, 3.7, 4.6, 3.9, 3.9, np.inf, 3.0],
    [3.3, 19.4, 1.5, 1.5, 3.1, 2.4, 2.4, 3.0, np.inf]
])


# ฟังก์ชันให้ผู้ใช้เลือกสถานที่
def user_select_places(places):
    selected_places = []
    print("กรุณาเลือกสถานที่จากรายการด้านล่าง (ป้อนหมายเลขที่ต้องการเลือก):")
    for i, place in enumerate(places):
        print(f"{i + 1}. {place}")

    print("ป้อนหมายเลขของสถานที่ที่ต้องการเลือก (คั่นด้วย comma ',' เช่น 1,2,3) หรือ 'q' เพื่อหยุด:")
    user_input = input("> ")

    if user_input.lower() == 'q':
        return selected_places

    indices = user_input.split(',')
    for index in indices:
        try:
            place_index = int(index.strip()) - 1
            if 0 <= place_index < len(places):
                selected_places.append(places[place_index])
        except ValueError:
            continue

    return selected_places


def main():
    while True:
        selected_places = user_select_places(places)

        if len(selected_places) < 2:
            print("กรุณาเลือกสถานที่อย่างน้อย 2 แห่งเพื่อคำนวณเส้นทาง")
            continue

        selected_distances, indices = get_distance_matrix(selected_places, places, distances)

        # เรียกใช้อัลกอริทึม Ant Colony Optimization
        ant_colony = AntColony(selected_distances, n_ants=10, n_best=5, n_iterations=50, decay=0.95, alpha=1, beta=1)
        shortest_path = ant_colony.run()

        # แสดงผลลัพธ์
        shortest_path_indices = shortest_path[0]
        print_path_details(shortest_path_indices, selected_places, selected_distances)

        # ถามผู้ใช้ว่าต้องการคำนวณอีกครั้งหรือไม่
        user_input = input("\nคุณต้องการเลือกสถานที่ใหม่หรือไม่? (y/n): ").strip().lower()
        if user_input != 'y':
            print("ขอบคุณที่ใช้โปรแกรม!")
            break


if __name__ == "__main__":
    main()


กรุณาเลือกสถานที่จากรายการด้านล่าง (ป้อนหมายเลขที่ต้องการเลือก):
1. สถาบันทักษิณคดีศึกษา
2. สวนสวา
3. วัดแหลมพ้อ
4. วัดโคกเปี้ยว
5. ศาลเจ้าไท้ก๋ง
6. วัดท้ายยอ
7. สำนักสงฆ์เขากุฎิ
8. วัดเขาบ่อ
9. ร้านค้า OTOP
ป้อนหมายเลขของสถานที่ที่ต้องการเลือก (คั่นด้วย comma ',' เช่น 1,2,3) หรือ 'q' เพื่อหยุด:


KeyboardInterrupt: Interrupted by user